In [1]:
import numpy as np
from parafoil import TurboRowPassage, TurboStagePassage, TurboMeshParameters
from parafoil import CamberThicknessAirfoil
from parafoil.passages.turbo import TurboRow3DPassage

mean_rotor_airfoil = CamberThicknessAirfoil(
    inlet_angle=np.radians(-60),
    outlet_angle=np.radians(-51),
    upper_thick_prop=[0.015, 0.05, 0.05, 0.015],
    lower_thick_prop=[0.015, 0.05, 0.05, 0.015],
    leading_prop=0.5,
    trailing_prop=0.5,
    chord_length=0.018*1000,
)
mean_rotor_pasage = TurboRowPassage(
    airfoil=mean_rotor_airfoil,
    spacing_to_chord=1.0,
    leading_edge_gap_to_chord=4,
    trailing_edge_gap_to_chord=4,
    type="camber",
    mesh_params=TurboMeshParameters(
        airfoil_label="wall1",
        bottom_label="periodic1",
        top_label="periodic2",
        inlet_label="inflow",
        outlet_label="outmix",
    )
)
mean_rotor_pasage.get_profile()
# hub_rotor_airfoil = mean_rotor_airfoil.clone(np.radians(-50), np.radians(-25))

hub_rotor_airfoil = CamberThicknessAirfoil(
    inlet_angle=np.radians(-50),
    outlet_angle=np.radians(-25),
    upper_thick_prop=[0.015, 0.05, 0.05, 0.015],
    lower_thick_prop=[0.015, 0.05, 0.05, 0.015],
    leading_prop=0.5,
    trailing_prop=0.5,
    chord_length=0.018*1000,
)
hub_rotor_passage = mean_rotor_pasage.clone(hub_rotor_airfoil)


tip_rotor_airfoil = CamberThicknessAirfoil(
    inlet_angle=np.radians(-67),
    outlet_angle=np.radians(-63),
    upper_thick_prop=[0.015, 0.05, 0.05, 0.015],
    lower_thick_prop=[0.015, 0.05, 0.05, 0.015],
    leading_prop=0.5,
    trailing_prop=0.5,
    chord_length=0.018*1000,
)
# tip_rotor_airfoil = mean_rotor_airfoil.clone(np.radians(-67),np.radians((-63)))
tip_rotor_passage = mean_rotor_pasage.clone(tip_rotor_airfoil)


blade = TurboRow3DPassage([hub_rotor_passage, mean_rotor_pasage, mean_rotor_pasage], radii=[0.1131*1000, 0.1697*1000, 0.2262*1000])

profile = blade.get_profile()

# import cadquery as cq
# cq.exporters.export(profile, "rotor67.step")

# import cadquery as cq
# cq.exporters.export(profile, "rotor67.step")

Overwriting auto display for cadquery Workplane and Shape


CadViewerWidget(anchor=None, cad_width=800, height=600, pinning=False, theme='light', title=None, tree_width=2…

In [2]:
import numpy as np
from parafoil import TurboRowPassage, TurboStagePassage, TurboMeshParameters
from parafoil import CamberThicknessAirfoil
from parafoil.passages.turbo import TurboRow3DPassage

mean_rotor_airfoil = CamberThicknessAirfoil(
    inlet_angle=np.radians(-60),
    outlet_angle=np.radians(-51),
    upper_thick_prop=[0.015, 0.05, 0.05, 0.015],
    lower_thick_prop=[0.015, 0.05, 0.05, 0.015],
    leading_prop=0.5,
    trailing_prop=0.5,
    chord_length=0.018*1000,
)
mean_rotor_pasage = TurboRowPassage(
    airfoil=mean_rotor_airfoil,
    spacing_to_chord=1.0,
    leading_edge_gap_to_chord=4,
    trailing_edge_gap_to_chord=4,
    type="camber",
    mesh_params=TurboMeshParameters(
        airfoil_label="wall1",
        bottom_label="periodic1",
        top_label="periodic2",
        inlet_label="inflow",
        outlet_label="outmix",
    )
)
mean_rotor_pasage.get_profile()
# hub_rotor_airfoil = mean_rotor_airfoil.clone(np.radians(-50), np.radians(-25))

hub_rotor_airfoil = CamberThicknessAirfoil(
    inlet_angle=np.radians(-50),
    outlet_angle=np.radians(-25),
    upper_thick_prop=[0.015, 0.05, 0.05, 0.015],
    lower_thick_prop=[0.015, 0.05, 0.05, 0.015],
    leading_prop=0.5,
    trailing_prop=0.5,
    chord_length=0.018*1000,
)
hub_rotor_passage = mean_rotor_pasage.clone(hub_rotor_airfoil)


tip_rotor_airfoil = CamberThicknessAirfoil(
    inlet_angle=np.radians(-67),
    outlet_angle=np.radians(-63),
    upper_thick_prop=[0.015, 0.05, 0.05, 0.015],
    lower_thick_prop=[0.015, 0.05, 0.05, 0.015],
    leading_prop=0.5,
    trailing_prop=0.5,
    chord_length=0.018*1000,
)
# tip_rotor_airfoil = mean_rotor_airfoil.clone(np.radians(-67),np.radians((-63)))
tip_rotor_passage = mean_rotor_pasage.clone(tip_rotor_airfoil)


blade = TurboRow3DPassage([hub_rotor_passage, mean_rotor_pasage, mean_rotor_pasage], radii=[0.1131*1000, 0.1697*1000, 0.2262*1000])

profile = blade.get_profile()
# pass
import cadquery as cq
cq.exporters.export(profile, "rotor67.step")

CadViewerWidget(anchor=None, cad_width=800, height=600, pinning=False, theme='light', title=None, tree_width=2…


*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
** WorkSession : Sending all data
 Step File Name : rotor67.step(15251 ents)  Write  Done


In [3]:
from meshql import GeometryQL, Split

with GeometryQL() as geo:
    (
        geo
        .load(
            profile,
            apply_splits= lambda split: (
                split
                # .from_plane(base_pnt=tuple(tip_rotor_passage.top_ctrl_pnts[1] + np.array([0.25, 0])*tip_rotor_passage.trailing_edge_gap), angle=(90, 0, 0))
                # .from_plane(base_pnt=tuple(hub_rotor_passage.top_ctrl_pnts[-2] - np.array([0.25, 0])*hub_rotor_passage.leading_edge_gap), angle=(90, 0, 0))
                # .from_faces("interior")
            )
        )
        # .vertices()
        # .setMeshSize(0.01)
        # .end()

        .setMeshAlgorithm("QuasiStructuredQuad")
        .generate(2)
        # .edges(type="split")
        .show("gmsh")
        # .end()
        # # .refine(1)
        # .setTransfiniteAuto(max_nodes=100, min_nodes=10)
        # .generate()
        # .show("gmsh")
    )

Info    : Build background mesh and guiding field ...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (BSpline)
Info    : [ 10%] Meshing curve 2 (BSpline)
Info    : [ 10%] Meshing curve 3 (BSpline)
Info    : [ 20%] Meshing curve 4 (BSpline)
Info    : [ 20%] Meshing curve 5 (BSpline)
Info    : [ 30%] Meshing curve 6 (BSpline)
Info    : [ 30%] Meshing curve 7 (BSpline)
Info    : [ 40%] Meshing curve 8 (BSpline)
Info    : [ 40%] Meshing curve 9 (BSpline)
Info    : [ 50%] Meshing curve 10 (BSpline)
Info    : [ 50%] Meshing curve 11 (BSpline)
Info    : [ 60%] Meshing curve 12 (BSpline)
Info    : [ 60%] Meshing curve 13 (BSpline)
Info    : [ 70%] Meshing curve 14 (BSpline)
Info    : [ 70%] Meshing curve 15 (BSpline)
Info    : [ 80%] Meshing curve 16 (BSpline)
Info    : [ 80%] Meshing curve 17 (BSpline)
Info    : [ 90%] Meshing curve 18 (BSpline)
Info    : [ 90%] Meshing curve 19 (BSpline)
Info    : [100%] Meshing curve 20 (BSpline)
Info    : [100%] Meshing curve 21 (BSpline)
Info   